In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler

In [2]:
path = "../collected_data/original_data"
dir_list = os.listdir(path)

In [3]:
# test reading
pd.read_csv(path + "/" + dir_list[0]).head()

,Unnamed: 0,vertices,redexes,redex_depths,redex_indexes,heights,widths,steps_time
0,0,53,3,4,12,15,21,131224
1,1,50,2,5,27,15,20,187277
2,2,27,1,1,1,8,10,32085
3,3,7,0,-1,-1,3,2,0


In [10]:
path_to_new_data = "../collected_data/data_for_lstm/"

dict_trans = dict()

all_data = pd.DataFrame()
for file_name in dir_list:
    data = pd.read_csv(path + "/" + file_name)
    data = data[[ri != -1 for ri in data["redex_indexes"]]]
    all_data = pd.concat([all_data, data]).reset_index(drop=True)

for col_name in ['vertices', 'redexes', 'redex_depths', 'heights', 'widths']:
    dict_trans[col_name] = PowerTransformer(method='yeo-johnson')
    dict_trans[col_name].fit(all_data[col_name].to_numpy().reshape(-1, 1))

dict_trans['steps_time'] = MinMaxScaler(feature_range=(0, 1))

all_data['steps_time'] = np.log(all_data['steps_time'])
dict_trans['steps_time'].fit(np.array(all_data['steps_time']).reshape(-1, 1))

for file_name in dir_list:
    data = pd.read_csv(path + "/" + file_name)
    data = data[[ri != -1 for ri in data["redex_indexes"]]]
    data = data.drop('Unnamed: 0', axis=1)

    for col_name in ['vertices', 'redexes', 'redex_depths', 'heights', 'widths']:
        norm_col_data = dict_trans[col_name].transform(np.array(data[col_name]).reshape(-1, 1))
        data[col_name] = list(norm_col_data.reshape(1, -1))[0]

    data['steps_time'] = np.log(data['steps_time'])
    norm_col_data = dict_trans['steps_time'].transform(np.array(data['steps_time']).reshape(-1, 1))
    data['steps_time'] = list(norm_col_data.reshape(1, -1))[0]


    if not os.path.exists(path_to_new_data + f"{len(data)}"):
        os.makedirs(path_to_new_data + f"{len(data)}")

    data.to_csv(path_to_new_data + f"{len(data)}/{file_name}", index=False)